In [ ]:
import os
import struct
import pandas as pd
import numpy as np
import talib as tdx

In [ ]:

def readTdxLdayFile(fname="sh000001.day"):
  dataSet=[]
  with open(fname,'rb') as fl:
    buffer=fl.read() #读取数据到缓存
    size=len(buffer) 
    rowSize=32 #通信达day数据，每32个字节一组数据
    code=os.path.basename(fname).replace('.day','')
    for i in range(0,size,rowSize): #步长为32遍历buffer
      row=list( struct.unpack('IIIIIfII',buffer[i:i+rowSize]) )
      row[1]=row[1]/100
      row[2]=row[2]/100
      row[3]=row[3]/100
      row[4]=row[4]/100
      row.pop() #移除最后无意义字段
      row.insert(0,code)
      dataSet.append(row) 

  data=pd.DataFrame(data=dataSet,columns=['code','tradeDate','open','high','low','close','amount','vol'])
#   data=data.set_index(['tradeDate'])
  return data


In [ ]:

def readTdxLdayPath(path):
  files = os.listdir(path)
  dataSet=[]
  for i in range(0,len(files)):
    fname = os.path.join(path,files[i])
    if os.path.isdir(fname):
      continue
    with open(fname,'rb') as fl:
      buffer=fl.read() #读取数据到缓存
      size=len(buffer) 
      rowSize=32 #通信达day数据，每32个字节一组数据
      code=os.path.basename(fname).replace('.day','')
      for i in range(0,size,rowSize): #步长为32遍历buffer
        row=list( struct.unpack('IIIIIfII',buffer[i:i+rowSize]) )
        row[1]=row[1]/100
        row[2]=row[2]/100
        row[3]=row[3]/100
        row[4]=row[4]/100
        row.pop() #移除最后无意义字段
        row.insert(0,code)
        dataSet.append(row) 

  data=pd.DataFrame(data=dataSet,columns=['code','tradeDate','open','high','low','close','amount','vol'])
  data=data.set_index(['code','tradeDate'])
  return data


In [ ]:
data_df=readTdxLdayPath('data') #读取目录下面的所有文件

In [ ]:
data_df.head()

In [73]:
def select1(code,data):
    # 连续三日缩量
    ch= data.close.iloc[-1] * 1.1
    cl= data.close.iloc[-1] * 0.9
#     ch= data.close * 1.1
#     cl = data.close * 0.9
    
    df=pd.concat([tdx.MA(data.close, x) for x in (5,10,20,30,60,90,120,250) ], axis = 1).dropna()[-1:]
    df.columns = [u'm5',u'm10',u'm20',u'm30',u'm60',u'm90',u'm120', u'm250']  
    df_h = df.apply(lambda x:x.max() <= ch,  axis = 1 )
    df_l = df.apply(lambda x:x.min() >= cl,  axis = 1 )
    
    df['dfh'] = df_h
    df['dfl'] = df_l
    df['code'] =code
#     out=df.iloc[-1].apply(lambda x: True if x>cl and x < ch else False)

    return df


In [ ]:
aaa=data_df.loc['sh000001',]

In [ ]:
aaa

In [75]:
select1('000001',aaa)

,m5,m10,m20,m30,m60,m90,m120,m250,dfh,dfl,code
tradeDate,,,,,,,,,,,
20190618,2895.974,2880.922,2886.1995,2892.297,3020.283833,2973.75,2867.055833,2796.51672,True,True,000001


In [ ]:
type(aaa)

In [ ]:
import talib as tdx

In [ ]:
tdx.MA(aaa.close,5)

In [ ]:
df=readTdxLdayFile()
df['mon'] = df.tradeDate.apply(lambda x : str(x)[0:6])
df=df.set_index(['tradeDate'])

In [ ]:
dfmax=df.groupby(['mon']).apply(lambda x: x[x.high ==x.high.max()])
dfmax.drop_duplicates(subset=['high','mon'],keep='first',inplace=True)
dfmin=df.groupby(['mon']).apply(lambda x: x[x.low ==x.low.min()])
dfmin.drop_duplicates(subset=['low','mon'],keep='first',inplace=True)

In [ ]:
dfmax.to_csv('max.csv')

In [ ]:
dfmin.to_csv('min.csv')

In [ ]:
dfmax

In [ ]:
for x in dfmax.index:
  print(df.loc[x[1]])